In [1]:
from lcls_live.datamaps.tabular import TabularDataMap, datamap_from_tao_data
from lcls_live.datamaps.klystron import KlystronDataMap, klystron_pvinfo, existing_LCLS_klystrons_sector_station, subbooster_pvinfo, SUBBOOSTER_SECTORS

from scipy.constants import e as e_charge
from pytao import Tao
import json
import os
from lcls_live import data_dir
import pandas as pd

# cu_hxr and cu_sxr

In [2]:
from lcls_live.datamaps.builders import *
    

## Utility function for creating map for given model 

In [3]:
def build_json_rep(model):
    #set up tao
    # Basic model with options
    MODEL = model
    OPTIONS = f'-noplot'
    INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'
    tao = Tao(INIT)
    
    
    rep = []
    
    
    dm = build_bpm_dm(tao, model)
    rep.append(
        {
        "name": "bpms",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    # Correctors
    dm = build_corrector_dm(tao)
    rep.append(
        {
        "name": "correctors",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    
    #subbooster_dm
    dm = build_subbooster_dm(model)
    rep.append(
        {
        "name": "subboosters",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #linac
    dm = build_linac_dm(model)
    rep.append(
        {
        "name": "linac",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #klystron
    dms = build_klystron_dms(tao, model)
    for dm in dms:
        rep.append(
            {
            "name": dm.name,
            "class": "klystron",
            "data": dm.to_json()
            }
        )
    
    
    #quad
    dm = build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #otr2
    dm = beginning_meas_twiss_datamap('WS02', 'WIRE:IN20:561' )
    rep.append(
        {
        "name": "beginning_WS02",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    
    #energy_meas
    dm = build_energy_dm(model)
    rep.append(
        {
        "name": "tao_energy_measurements",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    return rep

In [4]:
cu_hxr = build_json_rep("cu_hxr")
cu_sxr = build_json_rep("cu_sxr")

# cu_spec (i.e. the injector)

In [5]:
def build_cu_spec_linac_dm():
    dat =  [
    
        {'name': 'L0A_phase',
             'pvname':'ACCL:IN20:300:L0A_PDES', # deg
             'bmad_factor': 1/360,
             'bmad_name': 'L0A',
             'bmad_attribute': 'phi0'
            },
        {'name': 'L0B_phase',
             'pvname':'ACCL:IN20:400:L0B_PDES', # deg
             'bmad_factor': 1/360,
             'bmad_name': 'L0B',
             'bmad_attribute': 'phi0'
            },
        {'name': 'L0A_voltage',
             'pvname':'ACCL:IN20:300:L0A_ADES', # MV
             'bmad_factor': 1e6,
             'bmad_name': 'L0A',
             'bmad_attribute': 'voltage'
            },    
        {'name': 'L0B_voltage',
             'pvname':'ACCL:IN20:400:L0B_ADES', # MV
             'bmad_factor': 1e6,
             'bmad_name': 'L0B',
             'bmad_attribute': 'voltage'
            }
    ]
    df = pd.DataFrame(dat)
    dm = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor')
    return dm


In [6]:
def build_cu_spec_json_rep():
    #set up tao
    model = 'cu_spec'
    options = '-noplot'
    init = f'-init $LCLS_LATTICE/bmad/models/{model}/tao.init {options}'
    
    tao = Tao(init)
    
    
    rep = []
    
    # BPMs
    dm = build_bpm_dm(tao, model)
    rep.append(
        {
        "name": "bpms",
        "class": "tabular",
        "data": dm.to_json()
        }
    )     
    
    # Correctors
    dm = build_corrector_dm(tao)
    rep.append(
        {
        "name": "correctors",
        "class": "tabular",
        "data": dm.to_json()
        }
    )        
    
    # quad
    dm =  build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    # Linac
    dm = build_cu_spec_linac_dm()
    rep.append(
        {
        "name": "linac",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    return rep

cu_spec = build_cu_spec_json_rep()

# Dump all to master file

In [7]:
with open(os.path.join(data_dir, 'datamaps_master.json'), "w") as f:
    
    master = {"cu_hxr": cu_hxr,
              "cu_sxr": cu_sxr,
             "cu_spec": cu_spec}
    
    json.dump(master, f)

# Test

In [8]:
from lcls_live.datamaps import get_datamaps

DM = get_datamaps('cu_hxr')
DM.keys()

dict_keys(['bpms', 'correctors', 'subboosters', 'linac', 'K21_1', 'K21_2', 'K21_3', 'K21_4', 'K21_5', 'K21_6', 'K21_7', 'K21_8', 'K22_1', 'K22_2', 'K22_3', 'K22_4', 'K22_5', 'K22_6', 'K22_7', 'K22_8', 'K23_1', 'K23_2', 'K23_3', 'K23_4', 'K23_5', 'K23_6', 'K23_7', 'K23_8', 'K24_1', 'K24_2', 'K24_3', 'K24_4', 'K24_5', 'K24_6', 'K25_1', 'K25_2', 'K25_3', 'K25_4', 'K25_5', 'K25_6', 'K25_7', 'K25_8', 'K26_1', 'K26_2', 'K26_4', 'K26_5', 'K26_6', 'K26_7', 'K26_8', 'K27_1', 'K27_2', 'K27_3', 'K27_4', 'K27_5', 'K27_6', 'K27_7', 'K27_8', 'K28_1', 'K28_2', 'K28_3', 'K28_4', 'K28_5', 'K28_6', 'K28_7', 'K28_8', 'K29_1', 'K29_2', 'K29_3', 'K29_4', 'K29_5', 'K29_6', 'K29_7', 'K29_8', 'K30_1', 'K30_2', 'K30_3', 'K30_4', 'K30_5', 'K30_6', 'K30_7', 'K30_8', 'quad', 'beginning_WS02', 'tao_energy_measurements'])

In [9]:
DM['quad'].data

,bmad_name,pvname_rbv,pvname,bmad_factor,bmad_attribute
0,QA01,QUAD:IN20:361:BACT,QUAD:IN20:361:BDES,-0.925926,b1_gradient
1,QA02,QUAD:IN20:371:BACT,QUAD:IN20:371:BDES,-0.925926,b1_gradient
2,QE01,QUAD:IN20:425:BACT,QUAD:IN20:425:BDES,-0.925926,b1_gradient
3,QE02,QUAD:IN20:441:BACT,QUAD:IN20:441:BDES,-0.925926,b1_gradient
4,QE03,QUAD:IN20:511:BACT,QUAD:IN20:511:BDES,-0.925926,b1_gradient
...,...,...,...,...,...
186,QHXH47,QUAD:UNDH:4780:BACT,QUAD:UNDH:4780:BDES,-1.190476,b1_gradient
187,QUE1,QUAD:DMPH:300:BACT,QUAD:DMPH:300:BDES,-0.181818,b1_gradient
188,QUE2,QUAD:DMPH:380:BACT,QUAD:DMPH:380:BDES,-0.181818,b1_gradient
189,QDMP1,QUAD:DMPH:500:BACT,QUAD:DMPH:500:BDES,-0.232558,b1_gradient


In [10]:
DM['linac'].pvlist

['BMLN:LI21:235:MOTR',
 'BMLN:LI24:805:MOTR',
 'ACCL:LI21:1:L1S_S_PV',
 'ACCL:LI22:1:PDES',
 'ACCL:LI25:1:PDES']

In [11]:
MODEL = 'cu_spec'
OPTIONS = '-noplot'
INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'

In [12]:
tao = Tao(INIT)

In [13]:
datamap_from_tao_data(tao, 'orbit', 'x', tao_factor=.001, pv_attribute=':X').data

,pvname,tao_datum,tao_factor,bmad_name
0,BPMS:IN20:221:X,orbit.x[1],0.001,BPM2
1,BPMS:IN20:235:X,orbit.x[2],0.001,BPM3
2,BPMS:IN20:371:X,orbit.x[3],0.001,BPM5
3,BPMS:IN20:425:X,orbit.x[4],0.001,BPM6
4,BPMS:IN20:511:X,orbit.x[5],0.001,BPM8
5,BPMS:IN20:525:X,orbit.x[6],0.001,BPM9
6,BPMS:IN20:581:X,orbit.x[7],0.001,BPM10
7,BPMS:IN20:631:X,orbit.x[8],0.001,BPM11
8,BPMS:IN20:651:X,orbit.x[9],0.001,BPM12
9,BPMS:IN20:925:X,orbit.x[10],0.001,BPMS1
